In [13]:
# STEP 1: Getting the pre-trained Model

# Install necessary packages
! pip install --upgrade demucs
! pip install --upgrade torch
! pip install --upgrade torchaudio
! pip install --upgrade librosa
! pip install --upgrade musdb
! pip install --upgrade stempeg
! pip install numpy==2.0.0
! pip install --upgrade scipy
! pip install --upgrade scikit-learn

# Need to install fmpeg outside the console. Use conda install -c conda-forge ffmpeg

from demucs.pretrained import get_model

# Load the pre-trained Demucs model
model_name = "htdemucs"  # Change to the specific model name you wish to use
separator = get_model(model_name)

# Verifying the model is loaded
print(separator)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
BagOfModels(
  (models): ModuleList(
    (0): HTDemucs

In [12]:

# STEP 2: Normalize and Preprocess the Data

import os
import subprocess
import librosa
import numpy as np

# Function to extract audio stems from .mp4 file using ffmpeg
def extract_stems(mp4_path, extract_dir):
    stems = ['mixture', 'drums', 'bass', 'other', 'vocals']
    os.makedirs(extract_dir, exist_ok=True)
    for stem in stems:
        stem_path = os.path.join(extract_dir, f"{stem}.wav")
        command = ['ffmpeg', '-i', mp4_path, '-map', f'0:a:{stems.index(stem)}', stem_path]
        subprocess.run(command, check=True)
    print(f"Extracted stems from {mp4_path}")

# Normalize and convert audio to spectrogram functions
def normalize_audio(audio, orig_sr, target_sr=22050):
    audio_resampled = librosa.resample(audio, orig_sr=orig_sr, target_sr=target_sr)
    return audio_resampled

def audio_to_spectrogram(audio, sr=22050, n_fft=2048, hop_length=512):
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    return spectrogram

# Helper function to process and save spectrogram
def process_and_save(audio_path, save_spect_dir, target_sr=22050):
    audio, _ = librosa.load(audio_path, sr=None)
    audio_norm = normalize_audio(audio, target_sr)
    spectrogram = audio_to_spectrogram(audio_norm, sr=target_sr)
    
    # Create the directory if it doesn't exist
    os.makedirs(save_spect_dir, exist_ok=True)
    
    # Save the spectrogram as a numpy file
    spectrogram_path = os.path.join(save_spect_dir, os.path.basename(audio_path).replace('.wav', '.npy'))
    np.save(spectrogram_path, spectrogram)
    print(f"Saved spectrogram: {spectrogram_path}")

# Path to MUSDB18
musdb18_path = '../musdb18'
save_spect_path = '../outputs/spectrograms_musdb18'

# Process each song in the dataset
for subset in ['train', 'test']:
    subset_dir = os.path.join(musdb18_path, subset)
    save_subset_dir = os.path.join(save_spect_path, subset)
    
    for file in os.listdir(subset_dir):
        if file.endswith('.stem.mp4'):
            mp4_path = os.path.join(subset_dir, file)
            track_name = os.path.splitext(file)[0]
            extract_dir = os.path.join(subset_dir, track_name)
            
            # Extract audio stems
            extract_stems(mp4_path, extract_dir)
            
            # Process and save spectrograms for each stem
            for stem in ['mixture', 'drums', 'bass', 'other', 'vocals']:
                stem_path = os.path.join(extract_dir, f"{stem}.wav")
                if os.path.exists(stem_path):
                    process_and_save(stem_path, save_subset_dir)

print("Preprocessing complete.")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Extracted stems from ../musdb18/train/A Classic Education - NightOwl.stem.mp4


size=   29500kB time=00:02:51.22 bitrate=1411.4kbits/s speed= 471x    
video:0kB audio:29500kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000258%

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/mikea/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mik

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import